In [1]:
from langchain_huggingface import HuggingFaceEmbeddings


# Specify the model name and local directory for saving

local_model_path = "./huggingFaceModels/local_pubmedbert_model"

model_name = "neuml/pubmedbert-base-embeddings"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder=local_model_path
)



/home/mac/boltStation/pythonBackend/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import json


docs = []

with open('FrenchParameters.json', 'r') as file:
    docs = json.load(file)

print(docs)

[{'parameterName': 'Nombre de Globules Blancs (WBC), leucocytes, globules blancs', 'description': 'Mesure le nombre total de globules blancs dans le sang, qui sont essentiels pour la réponse immunitaire et la lutte contre les infections.'}, {'parameterName': 'Nombre de Globules Rouges (RBC), érythrocytes, hématies', 'description': "Compte le nombre de globules rouges, responsables du transport de l'oxygène des poumons vers les tissus et de l'élimination du dioxyde de carbone."}, {'parameterName': 'Hémoglobine (Hb), pigment sanguin, oxygène sanguin', 'description': "Détermine la quantité d'hémoglobine dans le sang, une protéine des globules rouges qui se lie à l'oxygène pour le transporter dans tout le corps."}, {'parameterName': 'Hématocrite (Hct), volume cellulaire, érythrocrit', 'description': "Mesure la proportion de globules rouges dans le sang, indiquant la viscosité sanguine et la capacité de transport d'oxygène."}, {'parameterName': 'Volume Corpusculaire Moyen (MCV), taille moye

In [4]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
import asyncio

url = "100.108.14.46"
client = QdrantClient(url)
collection_exists = client.collection_exists(
    collection_name="biologyParametersStore",
   
    )
if not collection_exists :
    client.create_collection(
        collection_name="biologyParametersStore",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE)
        )
    
vectors = []


    
for idx, doc in enumerate(docs):
    # Embed the 'content' property
    vector = await hf.aembed_query(doc["description"])
    
    # Add other properties as metadata in the payload
    payload = {
        "parameterName": doc['parameterName'],
        "description": doc['description']
    }
    vectors.append({'payload': payload, 'vector': vector, 'idx': idx})
    # Insert vector and metadata into Qdrant
print(vectors[0])

client.upsert(
    collection_name="biologyParametersStore",
    points=[
        PointStruct(id=vec['idx'], payload=vec['payload'], vector=vec['vector'])
        for vec in vectors
    ]
)



{'payload': {'parameterName': 'Nombre de Globules Blancs (WBC), leucocytes, globules blancs', 'description': 'Mesure le nombre total de globules blancs dans le sang, qui sont essentiels pour la réponse immunitaire et la lutte contre les infections.'}, 'vector': [0.19534932076931, 0.42014238238334656, 1.0006239414215088, 0.33125731348991394, -0.3762134313583374, 0.5261421203613281, -0.15396468341350555, -0.31231391429901123, 0.23576830327510834, 0.006206865888088942, -0.6261476874351501, 1.006616234779358, 0.23881889879703522, -0.04610385000705719, -0.7048378586769104, 0.38875851035118103, 0.6940615177154541, -0.9436666965484619, 0.354716956615448, -0.1794019490480423, -0.28710323572158813, -0.33421847224235535, -0.35036343336105347, -0.26038652658462524, 0.669126570224762, -0.682118833065033, 0.44721996784210205, -0.10739517211914062, 0.01037935446947813, 0.9276880621910095, -0.48941296339035034, 0.3337576985359192, -0.3162054121494293, 1.041182518005371, 0.0820254534482956, -0.2349395

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [6]:


query_text = "What is machine learning?"
query_vector = await hf.aembed_query('potatium')

results = client.search(
    collection_name="biologyParametersStore",
    query_vector=query_vector,
    limit=3,
    #score_threshold=0.5
)
formatted_results = [
    {
        "id": result.id,
        "score": result.score,
        "payload": result.payload,
    }
    for result in results
]
print(formatted_results)

[{'id': 8, 'score': 0.24637288, 'payload': {'parameterName': 'Volume Plaquettaire Moyen (MPV), taille moyenne des plaquettes, indice plaquettaire', 'description': 'Mesure la taille moyenne des plaquettes, fournissant des informations sur la production et la fonction des plaquettes.'}}, {'id': 106, 'score': 0.24298671, 'payload': {'parameterName': 'Niveau de prolactine, taux de prolactine', 'description': 'La prolactine est une hormone principalement impliquée dans la production de lait. Des niveaux élevés de prolactine peuvent affecter la fertilité, les cycles menstruels et la fonction sexuelle.'}}, {'id': 80, 'score': 0.20494151, 'payload': {'parameterName': 'Anticorps anti-HIV, test des anticorps HIV, test de dépistage des anticorps HIV', 'description': "Un test des anticorps pour le VIH, indiquant l'exposition au virus."}}]
